In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, decomposition, metrics
import os
import pandas as pd


In [39]:
""" df = pd.read_csv("data/creditcard.csv")

train_indices = np.random.choice(df.index, size = int(len(df) * 0.85), replace = False)
test_indices = df.index.difference(train_indices)

train = df.loc[train_indices]
test = df.loc[test_indices]

train.to_csv("data/train.csv", index = False)
test.to_csv("data/test.csv", index = False) """




' df = pd.read_csv("data/creditcard.csv")\n\ntrain_indices = np.random.choice(df.index, size = int(len(df) * 0.85), replace = False)\ntest_indices = df.index.difference(train_indices)\n\ntrain = df.loc[train_indices]\ntest = df.loc[test_indices]\n\ntrain.to_csv("data/train.csv", index = False)\ntest.to_csv("data/test.csv", index = False) '

In [40]:
# Read the train data
df = pd.read_csv("data/train.csv")

In [41]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Time,242085.0,94835.007758,47491.274141,0.000000,54243.000000,84727.000000,139340.000000,172792.000000
V1,242085.0,0.002638,1.958438,-56.407510,-0.918797,0.020464,1.316137,2.454930
V2,242085.0,0.000815,1.643492,-72.715728,-0.597816,0.064979,0.802582,21.467203
V3,242085.0,-0.001544,1.520658,-48.325589,-0.891138,0.180235,1.025291,4.226108
V4,242085.0,0.000670,1.416556,-5.683171,-0.849044,-0.018724,0.745564,16.715537
V5,242085.0,-0.000967,1.387046,-113.743307,-0.691943,-0.055914,0.612152,34.801666
V6,242085.0,0.002078,1.337085,-26.160506,-0.767640,-0.273696,0.400867,73.301626
V7,242085.0,-0.001002,1.247572,-41.506796,-0.554241,0.038475,0.569737,120.589494
V8,242085.0,0.000982,1.200456,-73.216718,-0.208499,0.022384,0.327352,20.007208
V9,242085.0,-0.000252,1.098348,-13.434066,-0.643775,-0.051490,0.598433,10.392889


In [43]:
df.corr()['Class']

Time     -0.012866
V1       -0.104456
V2        0.093795
V3       -0.198454
V4        0.136527
V5       -0.095825
V6       -0.045150
V7       -0.190534
V8        0.020354
V9       -0.099567
V10      -0.221541
V11       0.158500
V12      -0.266188
V13      -0.003831
V14      -0.309197
V15      -0.004877
V16      -0.200951
V17      -0.330278
V18      -0.113640
V19       0.036154
V20       0.019772
V21       0.045000
V22      -0.001230
V23      -0.003682
V24      -0.008909
V25       0.003910
V26       0.005931
V27       0.020230
V28       0.009339
Amount    0.005733
Class     1.000000
Name: Class, dtype: float64

## Supervised and Unsupervised Approaches
#### Logistic Regression and (PCA, Autoencoders)